In [132]:
import numpy as np
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister, execute, BasicAer, Aer
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram

# Define Gates

### Reverse CNOT

In [202]:
"""
Input: a, b

Output: 
P = a+b
Q = b
"""

c = QuantumCircuit(2)
c.h(0)
c.h(1)
c.cx(0,1)
c.h(0)
c.h(1)
revcx = c.to_gate(label="rev_CNOT")
c.draw()

┌───┐     ┌───┐
q_0: ┤ H ├──■──┤ H ├
     ├───┤┌─┴─┐├───┤
q_1: ┤ H ├┤ X ├┤ H ├
     └───┘└───┘└───┘

### Peres Gate

In [200]:
"""
Input: a, b, c

Output:
P = a
Q = a + b
R = a*b + c
"""
c = QuantumCircuit(3)
#c.reset(0)
#c.reset(1)
#c.reset(2)
#c.x(0)
#c.x(1)
#c.x(2)
c.ccx(0,1,2)
c.cx(0,1)
peres = c.to_gate(label="Peres")
c.draw()

q_0: ──■────■──
       │  ┌─┴─┐
q_1: ──■──┤ X ├
     ┌─┴─┐└───┘
q_2: ┤ X ├─────
     └───┘

In [201]:
#c.measure_all()
# Use Aer's qasm_simulator
simulator = Aer.get_backend('statevector_simulator')

# Execute the circuit on the qasm simulator
job = simulator.run(c)

# Grab results from the job
result = job.result()

# Return counts
outputstate = result.get_statevector(c, decimals=3)
print(outputstate)

Statevector([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j],
            dims=(2, 2, 2))


### Full Adder

In [162]:
"""
Input: 
a = 1st operand, 
b = 2nd operand, 
c = 0, 
d = c_in

Output: 
P = garbage, 
Q = garbage, 
R = sum, 
T = carry
"""
c = QuantumCircuit(4)
c.append(peres, [0,1,2])
c.swap(2,3)
c.append(peres, [1,2,3])
fa = c.to_gate(label="RFA")
c.draw()

┌────────┐             
q_0: ┤0       ├─────────────
     │        │   ┌────────┐
q_1: ┤1 Peres ├───┤0       ├
     │        │   │        │
q_2: ┤2       ├─X─┤1 Peres ├
     └────────┘ │ │        │
q_3: ───────────X─┤2       ├
                  └────────┘

### Fredkin

In [136]:
"""
By considering only the output qubit Q and by using input qubits 'a' as the selection line, Fredkin gate can 
be used as a 2x1 MUX.

Input: a, b, c

Output:
P = a,
Q = a'*b + a*c
R = a*b + a'*c
"""
c = QuantumCircuit(3)
c.append(revcx, [1,2])
c.ccx(0,1,2)
c.append(revcx, [1,2])
fredkin = c.to_gate(label="FG")
c.draw()

q_0: ───────────────■───────────────
     ┌───────────┐  │  ┌───────────┐
q_1: ┤0          ├──■──┤0          ├
     │  rev_CNOT │┌─┴─┐│  rev_CNOT │
q_2: ┤1          ├┤ X ├┤1          ├
     └───────────┘└───┘└───────────┘

# CSA Design-1

In [230]:
# TODO 1: Test the circuit
# TODO 2: Clean the code

# Input preparation
a_in = QuantumRegister(4, name="a")
b_in = QuantumRegister(4, name="b")
a_copies = QuantumRegister(4, name="a'")
b_copies = QuantumRegister(4, name="b'")
carries = QuantumRegister(8, name="c")
zero = QuantumRegister(2, name="zero")
c_in = QuantumRegister(2, name="c_in")
inputs = QuantumCircuit(a_in, b_in, a_copies, b_copies, carries, zero, c_in)
inputs.initialize("1100", a_in)
inputs.initialize("0011", b_in)
inputs.initialize("01010101", carries)
inputs.initialize("00", zero)
inputs.initialize("00", c_in)

# CSA
output = ClassicalRegister(5, name='output')
csa = QuantumCircuit(a_in, b_in, a_copies, b_copies, carries, zero, c_in, output)

# Initialize qubits
csa.cx(c_in[0], c_in[1])
for i in range(4):
    csa.cx(a_in[i], a_copies[i])
    csa.cx(b_in[i], b_copies[i])

# First stage
csa.append(fa, [a_in[0], b_in[0], zero[0], carries[0]])
csa.append(fa, [a_copies[0], b_copies[0], zero[1], carries[1]])
csa.append(fredkin, [c_in[1], zero[0], zero[1]])                    # mux for the sum qubits
csa.append(fredkin, [c_in[0], carries[0], carries[1]])              # mux for the carry qubits

csa.measure(zero[0], output[0])

csa.reset(zero[0])
csa.reset(zero[1])
csa.reset(c_in[0])      # Recycle c_in for the output of buffers
csa.reset(c_in[1])

csa.cx(carries[0], c_in[0])     # Buffer
csa.barrier()

# Second stage
csa.append(fa, [a_in[1], b_in[1], zero[0], carries[2]])
csa.append(fa, [a_copies[1], b_copies[1], zero[1], carries[3]])
csa.append(fredkin, [c_in[1], zero[0], zero[1]])
csa.append(fredkin, [c_in[0], carries[2], carries[3]])

csa.measure(zero[0], output[1])

csa.reset(zero[0])
csa.reset(zero[1])
csa.reset(c_in[0])  
csa.reset(c_in[1])

csa.cx(carries[2], c_in[0])
csa.barrier()

# Third stage
csa.append(fa, [a_in[2], b_in[2], zero[0], carries[4]])
csa.append(fa, [a_copies[2], b_copies[2], zero[1], carries[5]])
csa.append(fredkin, [c_in[1], zero[0], zero[1]])
csa.append(fredkin, [c_in[0], carries[4], carries[5]])

csa.measure(zero[0], output[2])

csa.reset(zero[0])
csa.reset(zero[1])
csa.reset(c_in[0])  
csa.reset(c_in[1])

csa.cx(carries[2], c_in[0])
csa.barrier()

# Fourth stage
csa.append(fa, [a_in[3], b_in[3], zero[0], carries[6]])
csa.append(fa, [a_copies[2], b_copies[2], zero[1], carries[7]])
csa.append(fredkin, [c_in[1], zero[0], zero[1]])
csa.append(fredkin, [c_in[0], carries[6], carries[7]])

csa.measure(zero[0], output[3])
csa.measure(carries[6],output[4])       # Measure also the carry out for the last stage

circuit = inputs + csa #+ output
circuit.draw()

/var/folders/0_/rj9sgsk533g7cn29_m9hndk00000gn/T/ipykernel_82895/3456383207.py:86: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  circuit = inputs + csa #+ output


┌──────────────────────┐                                       »
     a_0: ────┤0                     ├──────■────────────────────────────────»
              │                      │      │                                »
     a_1: ────┤1                     ├──────┼─────────■──────────────────────»
              │  Initialize(1,1,0,0) │      │         │                      »
     a_2: ────┤2                     ├──────┼─────────┼─────────■────────────»
              │                      │      │         │         │            »
     a_3: ────┤3                     ├──────┼─────────┼─────────┼─────────■──»
              ├──────────────────────┤      │         │         │         │  »
     b_0: ────┤0                     ├──────┼────■────┼─────────┼─────────┼──»
              │                      │      │    │    │         │         │  »
     b_1: ────┤1                     ├──────┼────┼────┼────■────┼─────────┼──»
              │  Initialize(0,0,1,1) │      │    │    │    │    │         │  »
     b_2: ────┤2                     ├──────┼────┼────┼────┼────┼────■────┼──»
              │                      │      │    │    │    │    │    │    │  »
     b_3: ────┤3                     ├──────┼────┼────┼────┼────┼────┼────┼──»
              └──────────────────────┘    ┌─┴─┐  │    │    │    │    │    │  »
    a'_0: ────────────────────────────────┤ X ├──┼────┼────┼────┼────┼────┼──»
                                          └───┘  │  ┌─┴─┐  │    │    │    │  »
    a'_1: ───────────────────────────────────────┼──┤ X ├──┼────┼────┼────┼──»
                                                 │  └───┘  │  ┌─┴─┐  │    │  »
    a'_2: ───────────────────────────────────────┼─────────┼──┤ X ├──┼────┼──»
                                                 │         │  └───┘  │  ┌─┴─┐»
    a'_3: ───────────────────────────────────────┼─────────┼─────────┼──┤ X ├»
                                               ┌─┴─┐       │         │  └───┘»
    b'_0: ─────────────────────────────────────┤ X ├───────┼─────────┼───────»
                                               └───┘     ┌─┴─┐       │       »
    b'_1: ───────────────────────────────────────────────┤ X ├───────┼───────»
                                                         └───┘     ┌─┴─┐     »
    b'_2: ─────────────────────────────────────────────────────────┤ X ├─────»
                                                                   └───┘     »
    b'_3: ───────────────────────────────────────────────────────────────────»
          ┌──────────────────────────────┐                                   »
     c_0: ┤0                             ├───────────────────────────────────»
          │                              │                                   »
     c_1: ┤1                             ├───────────────────────────────────»
          │                              │                                   »
     c_2: ┤2                             ├───────────────────────────────────»
          │                              │                                   »
     c_3: ┤3                             ├───────────────────────────────────»
          │  Initialize(0,1,0,1,0,1,0,1) │                                   »
     c_4: ┤4                             ├───────────────────────────────────»
          │                              │                                   »
     c_5: ┤5                             ├───────────────────────────────────»
          │                              │                                   »
     c_6: ┤6                             ├───────────────────────────────────»
          │                              │                                   »
     c_7: ┤7                             ├───────────────────────────────────»
          └─────┬──────────────────┬─────┘                                   »
  zero_0: ──────┤0                 ├─────────────────────────────────────────»
                │  Initialize(0,0) │                            

In [ ]:
#c.measure_all()
# Use Aer's qasm_simulator
simulator = Aer.get_backend('statevector_simulator')

# Execute the circuit on the qasm simulator
job = simulator.run(circuit)

# Grab results from the job
result = job.result()

# Return counts
outputstate = result.get_statevector(circuit, decimals=3)
print(outputstate)